# Файл с общим анализом данных и фрагментами функционала

### Загружаем данные

In [ ]:
import pandas as pd

categoryes = pd.read_csv("training-data/retail-rocket/category_tree.csv")
events = pd.read_csv("training-data/retail-rocket/events.csv")
properties1 = pd.read_csv("training-data/retail-rocket/item_properties_part1.csv")
properties2 = pd.read_csv("training-data/retail-rocket/item_properties_part2.csv")

events = events.sort_values(by="timestamp")
properties = pd.concat([properties1, properties2])

#data truncation
# categoryes = categoryes[0:1000]
# events = events[0:1000]
# properties = properties1[0:1000]

### Обрабатываем категории

In [ ]:
NONE = -1

sorted_categoryes = categoryes.fillna(NONE).sort_values(by="parentid")
sorted_categoryes["parentid"] = sorted_categoryes["parentid"].astype(int)
print(sorted_categoryes.dtypes)
sorted_categoryes.head()

#### Представляем категории в виде дерева

In [ ]:
from app.modules.categories import Tree, TreeNode


# заполняем словарь parent - childrens
category_map: dict[int, list[int]] = {}
for _, row in sorted_categoryes.iterrows():
    parentid = int(row['parentid'])
    categoryid = int(row['categoryid'])
    
    if parentid not in category_map:
        category_map[parentid] = []
    category_map[parentid].append(categoryid)

# заполняем дерево
categoryes_tree = Tree()

stack: list[TreeNode] = [categoryes_tree._root]
while len(stack) > 0:
    node = stack.pop()
    
    key = node.value if node.value != "" else NONE
    if key not in category_map:
        continue
        
    for child_value in category_map[key]:
        stack.append(categoryes_tree.append_to(node, child_value))



NameError: name 'sorted_categoryes' is not defined

In [ ]:
categoryes_tree._root.childrens

### Обрабатывем события

Находим уникальные события, чтобы создать EventEnum

In [ ]:
events["event"].unique()

находим последовательность действий для всех пользователей

In [ ]:
import numpy as np
from app.common.schemas import Event
from app.common.enums import EventEnum


users_events: dict[int, list[Event]] = {}
for _, user_event in events.iterrows():
    pyd_event = Event(
        event=EventEnum(user_event["event"]),
        user_id=user_event["visitorid"],
        item_id=user_event["itemid"],
        transaction_id=user_event["transactionid"] if not np.isnan(user_event["transactionid"]) else None,
        timestamp=user_event["timestamp"]
    )
    if users_events.get(user_event["visitorid"]) is None:
        users_events[user_event["visitorid"]] = [pyd_event]
    else:
        users_events[user_event["visitorid"]].append(pyd_event)

### Обрабатывем параметры предметов

```python
formalizable_properties: dict[
    int,  # property
    int   # индекс формализуемости: число_использований/число_всех_значений_параметра
]
```

In [ ]:
# удаляем параметры присущие только 5-ти процентам предметов
properties_counts = properties["property"].value_counts()
properties_counts = properties_counts[properties_counts > int(len(properties_counts)*0.95)]
properties_counts = properties_counts.index.tolist()

# создаём таблицу формализуемости параметров
stat_properties: dict[int, list[int, list[int]]] = {}
for _, item_property in properties.iterrows():
    if item_property["property"] not in properties_counts:
        continue
    if stat_properties.get(item_property["property"]) is None:
        stat_properties[item_property["property"]] = [1, [item_property["value"]]]
    elif item_property["value"] in stat_properties[item_property["property"]][1]:
        stat_properties[item_property["property"]][0] += 1
    else:
        stat_properties[item_property["property"]][0] += 1
        stat_properties[item_property["property"]][1].append(item_property["value"])

# находим наиболее формализуемые параметры
formalizable_properties = {prop: stat[0]/len(stat[1]) for prop, stat in stat_properties.items()}

# разделяем параметры по популярности
popular_properties = properties_counts[0:int(len(properties_counts)/3)]
standart_properties = properties_counts[int(len(properties_counts)/3):int(len(properties_counts)*2/3)]
unpopular_properties = properties_counts[int(len(properties_counts)*2/3):int(len(properties_counts))]

сохраняем полученные результаты

In [ ]:
import json
from pathlib import Path


with Path("training-data/combined/popular_properties.json").open("w") as f:
    json.dump(popular_properties, f)
with Path("training-data/combined/standart_properties.json").open("w") as f:
    json.dump(standart_properties, f)
with Path("training-data/combined/unpopular_properties.json").open("w") as f:
    json.dump(unpopular_properties, f)
with Path("training-data/combined/formalizable_properties.json").open("w") as f:
    json.dump(formalizable_properties, f)

создаём таблицу со всеми снимками срстояний параметров для каждого параметра
```python
item_properties: dict[
    int, # itemid
    dict[ # все property для itemid
        int, # property
        list[ # список всех состояний для property
            tuple[
                int, # timestamp
                str  # value
            ]
        ]
    ]
]
```

In [ ]:
item_properties: dict[int, dict[int, list[tuple[int, str]]]] = {}
item_avalable: dict[int, list[tuple[int, bool]]] = {}
item_category: dict[int, list[tuple[int, int]]] = {}
for _, values in properties.iterrows():
    if values["property"] == "categoryid":
        if item_category.get(values["itemid"]) is None:
            item_category[values["itemid"]] = [(values["timestamp"], values["value"])]
        else:
            item_category[values["itemid"]].append((values["timestamp"], values["value"]))
        continue

    if values["property"] == "available":
        if item_avalable.get(values["itemid"]) is None:
            item_avalable[values["itemid"]] = [(values["timestamp"], values["value"])]
        else:
            item_avalable[values["itemid"]].append((values["timestamp"], values["value"]))
        continue

    if item_properties.get(values["itemid"]) is None:
        item_properties[values["itemid"]] = {
            values["property"]: [(values["timestamp"], values["value"])]
        }
    elif item_properties[values["itemid"]].get(values["property"]) is None:
        item_properties[values["itemid"]][values["property"]] = [(values["timestamp"], values["value"])]
    else:
        item_properties[values["itemid"]][values["property"]].append((values["timestamp"], values["value"]))

сохраняем полученные результаты

In [ ]:
import json
import itertools
from pathlib import Path
from typing import Any

In [ ]:
with Path("training-data/combined/item_properties.json").open("w") as f:
    json.dump(item_properties, f)
with Path("training-data/combined/item_avalable.json").open("w") as f:
    json.dump(item_avalable, f)
with Path("training-data/combined/item_category.json").open("w") as f:
    json.dump(item_category, f)

In [ ]:
f1 = {
    key: [
        {
            'event': str(value.event),
            'user_id': value.user_id,
            'item_id': value.item_id,
            'transaction_id': value.transaction_id,
            'timestamp': value.timestamp.timestamp()
        } for value in values
    ] for key, values in users_events.items()
}
with Path("training-data/combined/users_events.json").open("w") as f:
    json.dump(f1, f)
short_users_events:dict[int, list[dict[str, Any]]]=dict(itertools.islice(f1.items(), int(len(f1)/10000)))
with Path("training-data/combined/short_users_events.json").open("w") as f:
    json.dump(short_users_events, f)
    
    

In [ ]:
with Path("training-data/combined/item_properties.json").open("r") as f:
    item_properties=dict(json.loads(f.read()))
with Path("training-data/combined/formalizable_properties.json").open("r") as f:
    formalizable_properties=dict(json.loads(f.read()))
with Path("training-data/combined/item_category.json").open("r") as f:
    item_category=dict(json.loads(f.read()))
with Path("training-data/combined/short_users_events.json").open("r") as f:
    short_users_events=dict(json.loads(f.read()))

In [ ]:
short_item_properties: dict[int, dict[int, list[tuple[int, str]]]] = {}
short_formalizable_properties: dict[int, float] = {}
short_item_category: dict[int, list[tuple[int, int]]] = {}
for events in short_users_events.values():
    for event in events:
        if short_item_properties.get(event["item_id"]) is None and item_properties.get(str(event["item_id"])) is not None:
            short_item_properties[event["item_id"]] = item_properties[str(event["item_id"])]
        if short_item_category.get(event["item_id"]) is None and item_category.get(str(event["item_id"])) is not None:
            short_item_category[event["item_id"]] = item_category[str(event["item_id"])]
for props in short_item_properties.values():
    for prop in props.keys():
        if short_formalizable_properties.get(prop) is None and formalizable_properties.get(str(prop)) is not None:
            short_formalizable_properties[prop] = formalizable_properties[str(prop)]

In [ ]:
with Path("training-data/combined/short_item_properties.json").open("w") as f:
    json.dump(short_item_properties, f)
with Path("training-data/combined/short_item_category.json").open("w") as f:
    json.dump(short_item_category, f) 
with Path("training-data/combined/short_formalizable_properties.json").open("w") as f:
    json.dump(short_formalizable_properties, f)

In [ ]:
short_item_category: dict[int, list[tuple[int, int]]] = {}
for events in short_users_events.values():
    for event in events:
        if (
            short_item_category.get(event["item_id"]) is None
            and short_item_category.get(event["item_id"]) is not None
            and formalizable_properties.get(event["item_id"]) is not None
        ):
            short_item_category[event["item_id"]] = item_category[event["item_id"]]
with Path("training-data/combined/short_item_category.json").open("w") as f:
    json.dump(short_item_category, f)

Используем сохраненные данные, очищая ОЗУ от предыдущих значений (нужно перезапустить ipynb)

In [1]:
import json
from pathlib import Path
from app.common.schemas import Event

with Path("training-data/combined/short_users_events.json").open("r") as f:
    short_users_events=dict(json.loads(f.read()))
users_events: dict[int, list[Event]] = {
    int(visitor_id): [
        Event.model_validate(event) for event in user_events
    ] for visitor_id, user_events in short_users_events.items()
}

with Path("training-data/combined/short_item_properties.json").open("r") as f:
    item_properties=dict(json.loads(f.read()))
with Path("training-data/combined/short_item_category.json").open("r") as f:
    item_category=dict(json.loads(f.read()))
with Path("training-data/combined/short_formalizable_properties.json").open("r") as f:
    short_formalizable_properties=dict(json.loads(f.read()))

        


In [ ]:
len(users_events)

140

In [2]:
from app.common.schemas import TrainingEvent
from app.common.enums import SeasonEnum

training_events: list[list[TrainingEvent]] =[]
for user_events in users_events.values():
    prev_event: Event | None = None
    training_user_events = []
    for user_event in sorted(user_events, key=lambda event: event.timestamp):
        training_event = TrainingEvent(
            event=user_event.event,
            season=SeasonEnum((user_event.timestamp.month % 12) // 3),
            timedelta_days=(user_event.timestamp - prev_event.timestamp).days if prev_event is not None else 0,
        )
        
        actual_timestamp: int = 0
        actual_category: int = 0
        if item_category.get(str(user_event.item_id)) is not None:
            for timestamp, category in item_category[str(user_event.item_id)]:
                if user_event.timestamp.timestamp() - timestamp < user_event.timestamp.timestamp() - actual_timestamp and user_event.timestamp.timestamp() - timestamp >= 0:
                    actual_timestamp = timestamp
                    actual_category = category
            training_event.item_category = actual_category
        
        form_factor = 0
        if item_properties.get(str(user_event.item_id)) is not None:
            for item_property, property_values in item_properties[str(user_event.item_id)].items():
                if short_formalizable_properties.get(item_property) is not None and short_formalizable_properties[item_property] > form_factor:
                    actual_timestamp: int = 0
                    actual_value: str = ""
                    for timestamp, values in property_values:
                        if user_event.timestamp.timestamp() - timestamp/1000 < user_event.timestamp.timestamp() - actual_timestamp and user_event.timestamp.timestamp() - timestamp/1000 >= 0:
                            actual_timestamp = int(timestamp/1000)
                            actual_value = values
                    if actual_value != "":
                        training_event.item_property = item_property
                        training_event.property_value = actual_value
                        form_factor = short_formalizable_properties[item_property]
        
        training_user_events.append(training_event)
        prev_event = user_event
    
    training_events.append(training_user_events)


In [ ]:
import random
training_events[random.randint(0, len(training_events)-1)]

[TrainingEvent(event=<EventEnum.VIEW: 'view'>, item_category=0, item_property=None, property_value=None, season=<SeasonEnum.SPRING: 1>, timedelta_days=None),
 TrainingEvent(event=<EventEnum.VIEW: 'view'>, item_category=0, item_property=None, property_value=None, season=<SeasonEnum.SPRING: 1>, timedelta_days=0)]

In [3]:
from app.modules.ml.model import create_gru_model
from app.common.enums import EventEnum

props: list[int | None] = []
values: list[str | None] = []
categories: list[int | None] = []

for user_training_event in training_events:
    for training_event in user_training_event:
        if training_event.item_property not in props:
            props.append(training_event.item_property)
        if training_event.property_value not in values:
            values.append(training_event.property_value)
        if training_event.item_category not in categories:
            categories.append(training_event.item_category)
    

num_propeties = len(props)
num_propet_values = len(values)
num_categories = len(categories)
num_actions = len(EventEnum)
model = create_gru_model(
    num_propeties = num_propeties,
    num_propet_values = num_propet_values,
    num_categories = num_categories,
    num_actions = num_actions,
    embedding_dim = 32,
    gru_units = 128,
    dropout_rate = 0.2
)

In [8]:
from app.modules.ml.model import prepare_data

all_actions: list[list[int]] = []
all_properties: list[list[int]] = []
all_values: list[list[str]] = []
all_categories: list[list[int]] = []
all_deltas: list[list[int]] = []
all_seasons: list[list[int]] = []

all_target_actions: list[list[int]] = []
all_target_properties: list[list[int]] = []
all_target_values: list[list[str]] = []
all_target_categories: list[list[int]] = []
for training_event in training_events:
    data = prepare_data(training_event)
    if data is not None:
        (
            actions,
            properties,
            values,
            categories,
            deltas,
            seasons,
            target_actions,
            target_properties,
            target_values,
            target_categories
        ) = data
        all_actions.extend(actions)
        all_properties.extend(properties)
        all_values.extend(values)
        all_categories.extend(categories)
        all_deltas.extend(deltas)
        all_seasons.extend(seasons)
        all_target_actions.append(target_actions)
        all_target_properties.append(target_properties)
        all_target_values.append(target_values)
        all_target_categories.append(target_categories)

In [6]:
from keras.layers import StringLookup

vocab: list[str] = []

for values in all_values:
    for value in values:
        if value is None:
            value = ""
        if value not in vocab:
            vocab.append(value)

for target_values in all_target_values:
    for target_value in target_values:
        if target_value is None:
            target_value = ""
        if target_value not in vocab:
            vocab.append(target_value)
    

lookup_layer = StringLookup(vocabulary=vocab, output_mode="int")
vectorized_values = lookup_layer(all_values)
target_vectorized_values = lookup_layer(all_target_values)


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [11]:
import numpy as np

# Приводим данные к целочисленному виду
all_actions = [
    np.array([
        int(action) for action in actions
    ])
    for actions in all_actions
]
all_properties = [
    np.array([
        int(action) for action in actions
    ])
    for actions in all_properties
]

all_target_properties = [
    np.array([
        int(action) for action in actions
    ])
    for actions in all_target_properties
]

In [12]:
from app.modules.ml.model import train_val_split
# Разделяем данные
train_inputs, val_inputs, train_targets, val_targets = train_val_split(
    [all_actions, all_properties, vectorized_values, all_categories, all_deltas, all_seasons],
    [all_target_actions, all_target_properties, target_vectorized_values, all_target_categories],
    val_split=0.2
)

NameError: name 'target_vectorized_values' is not defined

In [ ]:
# Обучаем модель
history = model.fit(
    train_inputs,
    train_targets,
    epochs=50,
    batch_size=32,
    validation_data=(val_inputs, val_targets),
    verbose=1,
)

ValueError: Unrecognized data type: x=[[array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0,